In [10]:
import os

# Check these before running

# KPUM Workstation
studydir = '/mnt/e/Research/Projects/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir
codedir = os.path.join('/home/radio/KPUM_NODDI','code/kpum_noddi')
datadirbasefolder = 'derivatives/dMRI_OrigProtocol'
protocol = 'ORIG' # 'NEW'
datadirbasefolder = 'derivatives/dMRI_np'
protocol = 'NEW' # 'NEW'
dPar = 1.7e-3


# Finn's Laptop
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir
codedir = os.path.join(studydir,'code/kpum_noddi')
codedir = os.path.join('/home/finn','Code/kpum_noddi')
datadirbasefolder = 'derivatives/dMRI_np'
protocol = 'NEW' # 'NEW'
datadirbasefolder = 'derivatives/dMRI_Testing7_dmri_pipeline'
protocol = 'NEW' # 'NEW'
dPar = 1.4e-3 # value to use in NODDI estimation

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerpath = os.path.join(studydir, datadirbasefolder)
subjecttrackerfile = 'Subject_Tracker_for_dmri_pipeline.tsv'

###################################################################################
# USER INPUT

# Participant details
subject = '035'
session = 'MR1'

In [11]:
import os

# Define I/O folders and files
niftifolder = os.path.join(studydir, niftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
derivatives = os.path.join(studydir,datadirbasefolder)
datadir = os.path.join(derivatives, os.path.join(f'sub-{subject}',f'ses-{session}'))
if not os.path.exists(datadir): # then make this directory
    os.makedirs(datadir)
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)

def perform_process(processcall) :
    import os, subprocess
    # Perform the process given by the processcall by launching into terminal
    p=subprocess.Popen(processcall, stdout=subprocess.PIPE, shell=True)
    # Poll process.stdout to show stdout live
    while True:
        output = p.stdout.readline()
        if p.poll() is not None:
            break
        if output:
            print(output.strip().decode("utf-8"))
    rc = p.poll()

In [14]:
########################################
## START

import os 
import pandas as pd

currdir = os.getcwd()
os.chdir(studydir)

## Process to perform - dmri_pipeline
process = 'dmri_pipeline'
processfile = 'dmri_pipeline.sh'
processfilepath = os.path.join(codedir,'shell', processfile)
processcall = f"bash {processfilepath} {subject} {session} {studydir} -d {derivatives} -dPar {dPar} -p {protocol}"
perform_process(processcall)

os.chdir(currdir)



KPUM NODDI dMRI pipeline
Subject:       	035
Session:        MR1
MRI Protocol:   ORIG
Derivatives:    derivatives/dMRI
TSV file:       derivatives/dMRI/Subject_Tracker_for_dmri_pipeline.tsv
dPar for NODDI: 0.0017
Threads:        4

/home/finn/Code/kpum_noddi/shell/dmri_pipeline.sh   	035 MR1 /mnt/s/Research/Projects/KPUM_NODDI/Data -d /mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI_Testing7_dmri_pipeline -dPar 0.0014 -p NEW
----------------------------

START - dmri_prepare_pipeline
Preparing for dMRI pipeline
Subject:           035
Session:           MR1
Session file:      nifti/sub-035/ses-MR1/session_QC.tsv
Data directory:    derivatives/dMRI/sub-035/ses-MR1

/home/finn/Code/kpum_noddi/shell/dmri_prepare_pipeline.sh       035 MR1 -d derivatives/dMRI/sub-035/ses-MR1
----------------------------
Running dmri_prepare_pipeline on subject 035 and session MR1

Transfer data in nifti/sub-035/ses-MR1/session_QC.tsv which has qc_pass_fail = 1 or 0.5
END - dmri_prepare_pipeline
Run

mrcalc: [  5%] computing: (dwi_den.mif - dwi_den_unr.mif)...dwifslpreproc: 
dwifslpreproc: Note that this script makes use of commands / algorithms that have relevant articles for citation; INCLUDING FROM EXTERNAL SOFTWARE PACKAGES. Please consult the help page (-help option) for more information.
dwifslpreproc: 
mrcalc: [ 28%] computing: (dwi_den.mif - dwi_den_unr.mif)...dwifslpreproc: Generated scratch directory: /mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI/sub-035/ses-MR1/dwi/preproc/dwifslpreproc/dwifslpreproc-tmp-S3GOQ8/
mrcalc: [ 30%] computing: (dwi_den.mif - dwi_den_unr.mif)...Command:  mrconvert /mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI/sub-035/ses-MR1/dwi/preproc/dwi_den_unr.mif /mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI/sub-035/ses-MR1/dwi/preproc/dwifslpreproc/dwifslpreproc-tmp-S3GOQ8/dwi.mif -json_export /mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI/sub-035/ses-MR1/dwi/preproc/dwifslpreproc/dwifslpreproc-tmp-S3GOQ8/dw

KeyboardInterrupt: 

Command:  eddy_cuda10.2 --imain=eddy_in.nii --mask=eddy_mask.nii --acqp=eddy_config.txt --index=eddy_indices.txt --bvecs=bvecs --bvals=bvals --slm=linear --repol --mporder=8 --s2v_niter=10 --s2v_interp=trilinear --s2v_lambda=1 --mbs_niter=20 --mbs_ksp=10 --mbs_lambda=10 --slspec=slspec.txt --out=dwi_post_eddy --verbose
Command:  eddy_cuda10.2 --imain=eddy_in.nii --mask=eddy_mask.nii --acqp=eddy_config.txt --index=eddy_indices.txt --bvecs=bvecs --bvals=bvals --slm=linear --repol --mporder=8 --s2v_niter=10 --s2v_interp=trilinear --s2v_lambda=1 --mbs_niter=20 --mbs_ksp=10 --mbs_lambda=10 --slspec=slspec.txt --out=dwi_post_eddy --verbose
